In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import cloudknot as ck
import diff_classifier.knotlets as kn
import numpy as np

In [2]:
#Files not to split
pups = ["P1"]
slices = ["S2"]
hemis = ["R"]
vids = np.linspace(10, 14, 14-10+1).astype(int)

exc = []
for pup in pups:
    for slic in slices:
        for hemi in hemis:
            for vid in vids:
                exc.append("'01_18_Experiment/{}/{}_{}_{}_{}.tif".format(pup, pup, slic, hemi, "%04d" % (vid,)))
exc

["'01_18_Experiment/P1/P1_S2_R_0010.tif",
 "'01_18_Experiment/P1/P1_S2_R_0011.tif",
 "'01_18_Experiment/P1/P1_S2_R_0012.tif",
 "'01_18_Experiment/P1/P1_S2_R_0013.tif",
 "'01_18_Experiment/P1/P1_S2_R_0014.tif"]

In [3]:
#Files to split
pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
hemis = ["L", "R"]
vids = np.linspace(0, 14, 14+1).astype(int)

to_split = []
for pup in pups:
    for slic in slices:
        for hemi in hemis:
            for vid in vids:
                name = "'01_18_Experiment/{}/{}_{}_{}_{}.tif".format(pup, pup, slic, hemi, "%04d" % (vid,))
                if name not in exc:
                    to_split.append(name)

In [4]:
github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
my_image = ck.DockerImage(func=kn.download_and_split, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

In [5]:
docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

cut1 = docker_string.find('# Create a default user')
cut2 = docker_string[cut1:].find('\n\n') + cut1
new_docker = docker_string[0:cut1]+docker_string[cut2:]
docker_overwrite = open(my_image.docker_path, 'w')
docker_overwrite.write(new_docker)
docker_overwrite.close()

In [7]:
knot = ck.Knot(name='download_and_track_with_my_image_5',
               docker_image = my_image,
               pars_policies=('AmazonS3FullAccess',))

TypeError: You must specify a directory to build in path

In [ ]:
knot.clobber